In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from moviepy.editor import VideoClip, ImageClip, concatenate_videoclips
import pandas as pd

runnersite = "https://runalyze.com/athlete/Schrottie"

# Funktion zum Abrufen der Webseite und Extrahieren der relevanten Informationen
def extract_activity_data():
    response = requests.get(runnersite)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    activities = []
    today = datetime.today().strftime('%d.%m')
    for row in soup.find_all('tr', class_='r training'):
        if 'icons8-Running' in row.find('td').text:
            cols = row.find_all('td')
            date_cell = cols[1].text.strip()
            date = date_cell.split()[0].replace('.', '')
            distance = cols[4].text.split()[0].replace(',', '.')
            duration = cols[5].text.strip()
            pace = cols[6].text.strip()
            if date == today:
                activities.append({'date': date, 'distance': distance, 'duration': duration, 'pace': pace})
    
    return activities



# Funktion zur Erstellung des Diagramms
def create_chart(activities):
    plt.figure(figsize=(8, 6))
    for activity in activities:
        plt.bar(activity['date'], float(activity['distance']), label='Distance: {} km'.format(activity['distance']))
    plt.xlabel('Date')
    plt.ylabel('Distance (km)')
    plt.title('Weekly Running Activities')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('chart.png')  # Speichern des Diagramms als PNG-Datei
    plt.close()

# Funktion zur Erstellung des Videos
def create_video(activities):
    video_clips = []
    for activity in activities:
        text_clip = ImageClip('chart.png').set_duration(3).resize(height=720)
        text_clip = text_clip.set_pos(('center', 'top'))
        text_clip = text_clip.set_start(0)
        
        title_clip = ImageClip('chart.png').set_duration(3).resize(height=720)
        title_clip = title_clip.set_pos(('center', 'top'))
        title_clip = title_clip.set_start(3)
        
        video_clip = concatenate_videoclips([text_clip, title_clip])
        video_clips.append(video_clip)
    
    final_clip = concatenate_videoclips(video_clips)
    final_clip.write_videofile("weekly_activities.mp4", fps=24, codec='libx264')

# Hauptfunktion
def main():
    activities = extract_activity_data()
    if activities:
        create_chart(activities)
        create_video(activities)
        print("Video wurde erfolgreich erstellt.")
    else:
        print("Keine Aktivitäten gefunden.")

if __name__ == "__main__":
    main()


Keine Aktivitäten gefunden.
